In [1]:
import torch

from collections import deque

import numpy as np

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import gym
import pandas as pd
from gym import spaces

In [2]:
pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00


In [15]:
import wandb
wandb.login()
run=wandb.init()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


training reward,▆███▄▇▆█▆███▄█▇█▇████▇▆██▇▅▆▁██▇██▇▇███▇
training reward moving average,████▂▄▄▃▄▄▃▄▄▃▄▄▄▅▄▂▂▁▃▄▄▅▄▃▂▂▂▂▃▃▂▂▁▂▃▄
training reward,-9.0
training reward moving average,-20.68687


In [16]:
from google.colab import drive
drive.mount('/content/drive')
dirname = '/content/drive/MyDrive/'
data=pd.read_csv(dirname+"transition_model_discrete_actions_90_min_history_rewards.csv",index_col=0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
# helper function to get index of state from data
def get_state_ind(df, state, s_cols):
        temp = df[pd.concat([df[s_col] == v for s_col, v in zip(s_cols, state)], axis=1).all(axis=1)]
        if temp.shape[0] > 0:
            return temp.sample(1).index[0]
        return -1

class CustomEnv(gym.Env):
    def __init__(self, dataframe, reward='simple_reward'):
        super(CustomEnv, self).__init__()
        self.df = dataframe

        feature_dim = int((max([int(col[4:]) for col in dataframe.columns if 'BG' in col]) + 1) / 2)

        self.s_cols = [f'BG_t{i}' for i in range(feature_dim)] + [f'HR_t{i}' for i in range(feature_dim)] + [f'IOB_t{i}' for i in range(feature_dim)]
        self.a_col = 'InsulinDelivered'
        self.s_prime_cols = [f'BG_t{i}' for i in range(feature_dim, 2*feature_dim)] + [f'HR_t{i}' for i in range(feature_dim, 2*feature_dim)] + [f'IOB_t{i}' for i in range(feature_dim, 2*feature_dim)]
        self.reward_col = reward

        actions = dataframe[self.a_col]
        self.action_space = spaces.Discrete(n=actions.nunique())

        bg_min = min([data[f'BG_t{i}'].min() for i in range(feature_dim*2)])
        bg_max = max([data[f'BG_t{i}'].max() for i in range(feature_dim*2)])
        hr_min = min([data[f'HR_t{i}'].min() for i in range(feature_dim*2)])
        hr_max = max([data[f'HR_t{i}'].max() for i in range(feature_dim*2)])
        iob_min = min([data[f'IOB_t{i}'].min() for i in range(feature_dim*2)])
        iob_max = max([data[f'IOB_t{i}'].max() for i in range(feature_dim*2)])

        self.observation_space = spaces.Box(low=np.array([bg_min]*feature_dim + [hr_min]*feature_dim + [iob_min]*feature_dim), high=np.array([bg_max]*feature_dim + [hr_max]*feature_dim + [iob_max]*feature_dim))
        self.current_state_ind = None


    def reset(self, seed=None):
        if seed:
            sample = self.df.sample(1, random_state=seed)
            s_ind = sample.index[0]
            s = sample[self.s_cols].values.flatten()
        else:
            sample = self.df.sample(1)
            s_ind = sample.index[0]
            s = sample[self.s_cols].values.flatten()
        self.current_state_ind = s_ind
        return s

    def step(self, action):
        # get the next state and reward based on the current state and action
        next_state_tx = self.df.iloc[self.current_state_ind]
        if next_state_tx[self.a_col] != action:
            raise Exception(f"Cannot take action (InsulinDelivered={action}) as this action doesn't exist for the current state at index {self.current_state_ind}!")
        next_state  = next_state_tx[self.s_prime_cols].values
        reward = next_state_tx[self.reward_col]
        done = self.is_done()
        info = {}

        self.current_state_ind = get_state_ind(self.df, next_state, self.s_cols)

        if self.current_state_ind == -1: # get closest state to next_state and update next_state
            self.current_state_ind = self.get_closest_state_ind(next_state)
            next_state = self.df.iloc[self.current_state_ind][self.s_cols].values

        return next_state, reward, done, info

    def does_action_exist(self, action):
        if action in self.df[self.a_col]:
            return True
        return False

    def get_closest_state_ind(self, state):
        distances = np.linalg.norm(state - self.df[self.s_cols].values, axis=1)
        return np.argsort(distances)[0]


    def get_closest_action(self, action):
        ind = np.random.choice(np.where((self.df[self.a_col] - action).abs() == (self.df[self.a_col] - action).abs().min())[0])
        return data.iloc[ind][self.a_col]

    def is_done(self):
      return False

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
data

,BG_t0,BG_t1,BG_t2,BG_t3,BG_t4,BG_t5,BG_t6,BG_t7,BG_t8,BG_t9,...,IOB_t28,IOB_t29,IOB_t30,IOB_t31,IOB_t32,IOB_t33,IOB_t34,IOB_t35,simple_reward,magni_reward
0,127.0,121.0,113.0,99.0,95.0,97.0,101.0,107.0,111.0,118.0,...,0.491872,0.502033,0.510965,0.531246,0.543538,0.555829,0.568121,0.580412,0,-10.211281
1,132.0,127.0,121.0,113.0,99.0,95.0,97.0,101.0,107.0,111.0,...,0.471303,0.491872,0.502033,0.510965,0.531246,0.543538,0.555829,0.568121,0,-12.579462
2,130.0,132.0,127.0,121.0,113.0,99.0,95.0,97.0,101.0,107.0,...,0.469787,0.471303,0.491872,0.502033,0.510965,0.531246,0.543538,0.555829,0,-12.139707
3,120.0,130.0,132.0,127.0,121.0,113.0,99.0,95.0,97.0,101.0,...,0.458479,0.469787,0.471303,0.491872,0.502033,0.510965,0.531246,0.543538,0,-12.800759
4,106.0,120.0,130.0,132.0,127.0,121.0,113.0,99.0,95.0,97.0,...,0.444016,0.458479,0.469787,0.471303,0.491872,0.502033,0.510965,0.531246,0,-14.602987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45712,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,-33.404175
45713,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,-33.404175
45714,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,-33.404175
45715,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,-33.404175


In [19]:
def get_action(df, state, env):
    state_ind = get_state_ind(df, state, env.s_cols)
    return df.iloc[state_ind][env.a_col]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
def run_target_update(Qprincipal, Qtarget):
    for v,v_ in zip(Qprincipal.model.parameters(), Qtarget.model.parameters()):
        v_.data.copy_(v.data)

In [21]:
env = CustomEnv(data)
observation = env.reset()
#print(observation)
# action = env.action_space.sample()
action = get_action(data, observation, env)
next_observation, reward, done, info = env.step(action)

/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [22]:
class Policy(object):

    def __init__(self, obssize, actsize, lr):
        """
        obssize: size of the states
        actsize: size of the actions
        """
        # TODO DEFINE THE MODEL
        self.model = torch.nn.Sequential(torch.nn.Linear(obssize, 256),
                                         torch.nn.ReLU(),
                                         torch.nn.Linear(256, 128),
                                         torch.nn.ReLU(),
                                         torch.nn.Linear(128, actsize),
                                         )
        # DEFINE THE OPTIMIZER
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

        # RECORD HYPER-PARAMS
        self.obssize = obssize
        self.actsize = actsize

        # TEST
        self.compute_prob(np.random.randn(obssize).reshape(1, -1))

    def compute_prob(self, states):
        """
        compute prob distribution over all actions given state: pi(s)
        states: numpy array of size [numsamples, obssize]
        return: numpy array of size [numsamples, actsize]
        """
        states = torch.FloatTensor(states)
        prob = torch.nn.functional.softmax(self.model(states), dim=-1)
        return prob.cpu().data.numpy()

    def _to_one_hot(self, y, num_classes):
        """
        convert an integer vector y into one-hot representation
        """
        scatter_dim = len(y.size())
        y_tensor = y.view(*y.size(), -1)
        zeros = torch.zeros(*y.size(), num_classes, dtype=y.dtype)
        return zeros.scatter(scatter_dim, y_tensor, 1)

    def train(self, states, actions, Qs):
        """
        states: numpy array (states)
        actions: numpy array (actions)
        Qs: numpy array (Q values)
        """
        states = torch.FloatTensor(states)
        actions = torch.LongTensor(actions)
        Qs = torch.FloatTensor(Qs)

        # COMPUTE probability vector pi(s) for all s in states
        logits = self.model(states)
        prob = torch.nn.functional.softmax(logits, dim=-1)

        # Compute probaility pi(s,a) for all s,a
        action_onehot = self._to_one_hot(actions, actsize)
        prob_selected = torch.sum(prob * action_onehot, axis=-1)

        # FOR ROBUSTNESS
        prob_selected += 1e-8

        loss = -torch.mean(torch.log(prob_selected) * Qs)

        # BACKWARD PASS
        self.optimizer.zero_grad()
        loss.backward()

        # UPDATE
        self.optimizer.step()

        return loss.detach().cpu().data.numpy()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
class ValueFunction(object):

    def __init__(self, obssize, lr):
        """
        obssize: size of states
        """
        self.model = torch.nn.Sequential(torch.nn.Linear(obssize, 256),
                                         torch.nn.ReLU(),
                                         torch.nn.Linear(256, 128),
                                         torch.nn.ReLU(),
                                         torch.nn.Linear(128, 1),
                                         )

        # DEFINE THE OPTIMIZER
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

        # RECORD HYPER-PARAMS
        self.obssize = obssize
        self.actsize = actsize

        # TEST
        self.compute_values(np.random.randn(obssize).reshape(1, -1))

    def compute_values(self, states):
        """
        compute value function for given states
        states: numpy array of size [numsamples, obssize]
        return: numpy array of size [numsamples]
        """
        states = torch.FloatTensor(states)
        return self.model(states).cpu().data.numpy()

    def train(self, states, targets):
        """
        states: numpy array
        targets: numpy array
        """
        states = torch.FloatTensor(states)
        targets = torch.FloatTensor(targets)

        # COMPUTE Value PREDICTIONS for states
        v_preds = self.model(states)

        # LOSS
        loss = torch.mean(torch.square(v_preds - targets))

        # BACKWARD PASS
        self.optimizer.zero_grad()
        loss.backward()

        # UPDATE
        self.optimizer.step()

        return loss.detach().cpu().data.numpy()

In [24]:
def discounted_rewards(r, gamma):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_sum = 0
    for i in reversed(range(0,len(r))):
        discounted_r[i] = running_sum * gamma + r[i]
        running_sum = discounted_r[i]
    return list(discounted_r)

In [ ]:
%%wandb



# parameter initializations (you can change any of these)
alpha = 1e-2  # learning rate for PG
beta = 1e-3  # learning rate for baseline
numtrajs = 5  # num of trajecories from the current policy to collect in each iteration
iterations = 100  # total num of iterations
gamma = .99  # discount

# initialize environment
obssize = env.observation_space.low.size
actsize = env.action_space.n

# initialize networks
actor = Policy(obssize, actsize, alpha)  # policy initialization: IMPORTANT: this is the policy you will be scored on
baseline = ValueFunction(obssize, beta)  # baseline initialization

insulin_delivered_to_action_mapper = {data['InsulinDelivered'].unique()[i]: i for i in range(data['InsulinDelivered'].nunique())}
action_to_insulin_delivered_mapper = {v: k for k, v in insulin_delivered_to_action_mapper.items()}

run = wandb.init(
    # set the wandb project where this run will be logged
    project="rl-based-artificial-pancreas",

    # track hyperparameters and run metadata
    config={
    "method": "policy-gradient",
    "alpha": alpha,
    "beta": beta,
    "num_trajs": numtrajs,
    "iterations": iterations,
    "gamma": gamma,
    }
)

#To record training reward for logging and plotting purposes
rrecord = []

# main iteration
for ite in range(iterations):
    print(ite)
    # To record trajectories generated from current policy
    OBS = []  # observations
    ACTS = []  # actions
    ADS = []  # advantages (to compute policy gradient)
    VAL = []  # Monte carlo value predictions (to compute baseline, and policy gradient)

    for num in range(numtrajs):
        # To keep a record of states actions and reward for each episode
        obss = []  # states
        acts = []   # actions
        rews = []  # instant rewards

        obs = env.reset()
        done = False

        # TODO: run one episode using the current policy "actor"
        for i in range(50):
            prob = actor.compute_prob(np.expand_dims(obs, 0))
            prob /= np.sum(prob)  # normalizing again to account for numerical errors
            action = get_action(data, obs, env)    # np.ndarray.item(np.random.choice(actsize, p=prob.flatten(), size=1))   choose according to the distribution prob
            obss.append(obs)
            acts.append(insulin_delivered_to_action_mapper[action])
            obs, reward, done, info = env.step(action)
            rews.append(reward)

        # TODO: record all observations (states, actions, rewards) from the epsiode in  obss, acts, rews
        OBS.extend(obss)
        ACTS.extend(acts)
        #Below is for logging training performance
        rrecord.append(np.sum(rews))

        # TODO:  Use discounted_rewards function to compute \hat{V}s/\hat{Q}s  from instant rewards in rews
        discounted_rewards_list = discounted_rewards(rews, gamma)
        VAL.extend(discounted_rewards_list)
        # TODO: record the computed \hat{V}s in VAL, states obss in OBS, and actions acts in ACTS, for batch update

    # AFTER collecting numtrajs trajectories:

    #1. TODO: train baseline
    baseline.train(np.array(OBS), np.array(VAL))
    """
        Use the batch (OBS, VAL) of states and value predictions as targets to train baseline.S
        Use baseline.train : note that this takes as input numpy array, so you may have to convert
        lists into numpy array using np.array()
    """

    # 2.TODO: Update the policy
    baselines = baseline.compute_values(np.array(OBS))
    advantages = np.array(VAL) - baselines
    actor.train(np.array(OBS), np.array(ACTS), advantages)
    """
        Compute baselines: use basline.compute_values for states in the batch OBS
        Compute advantages ADS using VAL and computed baselines
        Update policy using actor.train using OBS, ACTS and ADS
    """




    #printing moving averages for smoothed visualization.
    #Do not change below: this assume you recorded the sum of rewards in each episide in the list rrecord
    fixedWindow=100
    movingAverage=0
    if len(rrecord) >= fixedWindow:
        movingAverage=np.mean(rrecord[len(rrecord)-fixedWindow:len(rrecord)-1])

    #wandb logging
    wandb.log({ "training reward" : rrecord[-1], "training reward moving average" : movingAverage})

training reward,█▁
training reward moving average,▁▁
training reward,-69.0
training reward moving average,0


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65


In [25]:
# DO NOT CHANGE CODE HERE

### DO NOT CHANGE
def evaluate(policy, env, episodes):
    # main iteration
    score = 0
    for episode in range(episodes):
        obs = env.reset()
        done = False
        rsum = 0

        while not done:
            p = policy.compute_prob(np.expand_dims(obs,0)).ravel()
            p /= np.sum(p)
            states=env.reset()
            action = get_action(data, states, env)

            # env stepping forward
            newobs, r, done, _ = env.step(action)

            # update data
            rsum += r
            obs = newobs
            # temp=data.loc[0,[f'BG_t{i}' for i in range(18,36)]].values.tolist()
            done=all(obs[i]>90 and obs[i]<120 for i in range(17))


        wandb.log({"eval reward" : rsum})
        score +=rsum
    score = score/episodes

    return score

In [26]:
# DO NOT CHANGE CODE HERE
# after training, we will evaluate the performance of the learned policy "actor"
# on a target environment
env_test = CustomEnv(data)
eval_episodes = 1000
score = evaluate(actor, env_test, eval_episodes)
wandb.run.summary["score"]=score
print("eval performance of the learned policy: {}".format(score))

/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


eval performance of the learned policy: -1.484


In [ ]:
run.finish()